### This note book runs the efficent model using the noisy students weights. We will use the features and fine tune it with our dataset.
### we do not have enough data for training it from scratch

In [1]:
## NO HUB IMPORT PLEASE USE THIS WITH TENSORFLOW 2.xx nightly
import os
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow import keras
import numpy as np
import pathlib
import cv2
import PIL
import PIL.Image
import time
import datetime
import matplotlib.pylab as plt
import tensorboard
SEEDS = 42
np.random.seed(SEEDS)
tf.random.set_seed(SEEDS)
import itertools
print("TF version:", tf.__version__)
GPU = tf.config.list_physical_devices('GPU')
CPU= tf.config.list_physical_devices('CPU')
print("Num GPUs:", len(GPU))

print("Num CPUs:", len(CPU))

TF version: 2.9.0-dev20220113
Num GPUs: 1
Num CPUs: 1


2022-01-26 16:53:32.890297: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-26 16:53:32.900227: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-26 16:53:32.900799: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


### set the memory growth incremental

In [2]:
## The logical GPUs is not supported in the tensflow available binary, you need to compile it from scratch for multiple logical GPUs

gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
      tf.config.experimental.set_memory_growth(gpus[0], True)
      print(len(gpus), "Physical GPUs,")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

1 Physical GPUs,


### defining training dataset diredtories

In [ ]:
data_dir_1= "/home/pms/PMS-dataset/data-set/10-class/crop/training"
data_dir_2= "/home/pms/PMS-dataset/data-set/10-class/output/training"

data_dir_1 = pathlib.Path(data_dir_1)
data_dir_2 = pathlib.Path(data_dir_2)
print("Training-Directory-combined: ",data_dir_1)
print("Training-Directory-only-road: ",data_dir_1)


image_count_1 = len(list(data_dir_1.glob('*/*.jpg')))
image_count_2 = len(list(data_dir_2.glob('*/*.jpg')))
print(image_count_1)
print(image_count_2)

### preparing dataset from the directory into training and validation data

In [ ]:
IMAGE_WIDTH = 512
IMAGE_HEIGHT = 512
IMAGE_SIZE = (IMAGE_WIDTH, IMAGE_HEIGHT)
BATCH_SIZE = 8
TOTAL_CLASSES = 10

In [ ]:
train_ds_1 = tf.keras.utils.image_dataset_from_directory(
  str(data_dir_1),
  validation_split=0.2,
  subset="training",
  seed=SEEDS,
  image_size=IMAGE_SIZE,
  batch_size=BATCH_SIZE
)

train_ds_2 = tf.keras.utils.image_dataset_from_directory(
  str(data_dir_2),
  validation_split=0.2,
  subset="training",
  seed=SEEDS,
  image_size=IMAGE_SIZE,
  batch_size=BATCH_SIZE
)


val_ds_1 = tf.keras.preprocessing.image_dataset_from_directory(
  str(data_dir_1),
  validation_split=0.2,
  subset="validation",
  seed=SEEDS,
  image_size=IMAGE_SIZE,
  batch_size=BATCH_SIZE
)

val_ds_2 = tf.keras.preprocessing.image_dataset_from_directory(
  str(data_dir_2),
  validation_split=0.2,
  subset="validation",
  seed=SEEDS,
  image_size=IMAGE_SIZE,
  batch_size=BATCH_SIZE
)


class_names = np.array(train_ds_1.class_names)
print(class_names)
class_names = np.array(train_ds_2.class_names)
print(class_names)
class_names_val = np.array(val_ds_1.class_names)
print(class_names_val)
class_names_val = np.array(val_ds_2.class_names)
print(class_names_val)
#concatenate the datasets
train_ds = train_ds_1.concatenate(train_ds_2)
val_ds = val_ds_1.concatenate(val_ds_2)

train_ds = train_ds_1.concatenate(train_ds_2)
val_ds = val_ds_1.concatenate(val_ds_2)

print(len(list(train_ds_1.as_numpy_iterator())))
print(len(list(train_ds_2.as_numpy_iterator())))
print(len(list(train_ds.as_numpy_iterator())))

### Apply Data augmentation to the training set and rescaling the images to 0-1 in both training and validation set and autotune

In [ ]:
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.RandomFlip('horizontal'),
  tf.keras.layers.RandomFlip('vertical'),
  tf.keras.layers.RandomRotation(2),
  tf.keras.layers.RandomZoom(.5),
  tf.keras.layers.RandomContrast(0.1, 0.2)
])
# apply normalization and augmentation to the training set and validation set
############################################################################
#normalization_layer = tf.keras.layers.Rescaling(1./255)
#train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y)) # Where x—images, y—labels.
#val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y)) # Where x—images, y—labels.
############################################################################
train_ds = train_ds.map(lambda x, y: (data_augmentation(x), y))

AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
# check the batch size
image_batch, label_batch = next(iter(train_ds.take(1)))
print(image_batch.shape, label_batch.shape)

In [ ]:
def plot_hist(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']

    loss = history.history['loss']
    val_loss = history.history['val_loss']

    plt.figure(figsize=(8, 8))
    plt.subplot(2, 1, 1)
    plt.plot(acc, label='Training Accuracy')
    plt.plot(val_acc, label='Validation Accuracy')
    plt.legend(loc='lower right')
    plt.ylabel('Accuracy')
    plt.ylim([min(plt.ylim()),1.2])
    plt.title('Training and Validation Accuracy')

    plt.subplot(2, 1, 2)
    plt.plot(loss, label='Training Loss')
    plt.plot(val_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.ylabel('Cross Entropy')
    plt.ylim([0,10.0])
    plt.title('Training and Validation Loss')
    plt.xlabel('epoch')
    plt.show()

In [ ]:
## for debugging purpose
#tf.debugging.set_log_device_placement(True) # this line is to turn on debugging information

train_callbacks = [
    keras.callbacks.EarlyStopping(
        monitor="val_accuracy", patience=3, restore_best_weights=True
    )
]

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

tensorboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir=log_dir,
    histogram_freq=1) # Enable histogram computation for every epoch.

In [ ]:

from tensorflow.keras.applications import EfficientNetV2L
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
base_learning_rate = 0.005

def build_model(num_classes,base_learning_rate):
    inputs = layers.Input(shape=(IMAGE_HEIGHT, IMAGE_WIDTH, 3))
    
    model = EfficientNetV2L(include_top=False,input_tensor=inputs, weights="imagenet")

    # Freeze the pretrained weights
    model.trainable = False

    # Rebuild top
    x = layers.GlobalAveragePooling2D(name="avg_pool")(model.output)
    x = layers.BatchNormalization()(x)

    top_dropout_rate = 0.2
    x = layers.Dropout(top_dropout_rate, name="top_dropout")(x)
    outputs = layers.Dense(TOTAL_CLASSES, activation="softmax", name="pred")(x)
    #outputs = layers.Dense(TOTAL_CLASSES, name="pred")(x) # DEFAULT ACTIVATION IS LINEAR
    # Compile
    model = tf.keras.Model(inputs, outputs, name="EfficientNetV2L")

    model.compile(
      optimizer = tf.keras.optimizers.SGD(learning_rate=base_learning_rate,momentum=0.9), 
      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
      metrics=['accuracy'])

    return model

model = build_model(num_classes=TOTAL_CLASSES,base_learning_rate=base_learning_rate)
model.summary()



In [ ]:
## training the transfer learning model
epochs = 5 # @param {type: "slider", min:8, max:80}
# class weight are calculated by the total of samples divided by the number of classes and the number of samples in each class
class_weights = {0: 0.85,1 : 0.83,2 : 1.495,3 : 1.57,4 : 2.153,5 : 2.53,6 : 1.17,7 : 0.89,8 : 0.609,9 : 0.54}
history = model.fit(
        train_ds,
        epochs=epochs,
        validation_data=val_ds,
        callbacks=tensorboard_callback,
        class_weight=class_weights
    )
    
plot_hist(history)

In [ ]:
##for fine tuning we keep a accurate optimizer like SGD and slow learning rate
learning_rate = 0.0005
def unfreeze_model(model,learning_rate):
    # We unfreeze the top 20 layers while leaving BatchNorm layers frozen
    for layer in model.layers[-20:]:
        if not isinstance(layer, layers.BatchNormalization):
            layer.trainable = True

    optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=0.9)
    model.compile(
    optimizer = optimizer, 
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'])


unfreeze_model(model,learning_rate)
model.summary()


In [ ]:
##training the fine tuned model
NUM_EPOCHS = 50
# class weight are calculated by the total of samples divided by the number of classes and the number of samples in each class
class_weights = {0: 0.85,1 : 0.83,2 : 1.495,3 : 1.57,4 : 2.153,5 : 2.53,6 : 1.17,7 : 0.89,8 : 0.609,9 : 0.54}
history = model.fit(
        train_ds,
        epochs=NUM_EPOCHS,
        validation_data=val_ds,
        callbacks=tensorboard_callback,
        class_weight=class_weights
    )

plot_hist(history)

In [ ]:
reloaded = model
with tf.device('device:GPU:0'):
  test_data_dir = pathlib.Path('/home/pms/PMS-dataset/data-set/10-class/output/testing')
  print(test_data_dir)
  img_height = 512
  img_width = 512
  batch_size = 8
  class_number = 10

  test_dataset = tf.keras.utils.image_dataset_from_directory(
    test_data_dir,
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)
   ## NO NEED for normalization here 
  
  #test_dataset = test_dataset.map(lambda x, y: (data_augmentation (x), y)) # Where x—images, y—labels.
  ds = train_ds.concatenate(val_ds).concatenate(test_dataset)
  loss, accuracy = reloaded.evaluate(test_dataset)
  print('Test accuracy :', accuracy)
  print('Test loss :', loss)

/home/pms/PMS-dataset/data-set/10-class/combined/testing
Found 292 files belonging to 10 classes.
37/37 [==============================] - 12s 329ms/step - loss: 4.3840 - accuracy: 0.2842
37/37 [==============================] - 12s 329ms/step - loss: 5.3218 - accuracy: 0.2808
37/37 [==============================] - 12s 332ms/step - loss: 4.7976 - accuracy: 0.3185

In [ ]:
model.save('pms_model/efficientnetv2l-finetune-imagenet-512-new')


validation accuracy : 0.6344085931777954
Training accuracy : 0.9576897025108337
Test accuracy : 0.3116438388824463 292
Total accuracy : 0.8142127394676208


In [ ]:
import xlwt
from xlwt import Workbook
  
# Workbook is created
wb = Workbook()
# add_sheet is used to create sheet.
test = wb.add_sheet('Test-Summary')

In [ ]:
with tf.device('device:GPU:0'):
    result_test_data_dir = pathlib.Path('/home/pms/PMS-dataset/data-set/10-class/output/testing')
    output_test_dir = pathlib.Path('/home/pms/PMS-dataset/data-set/10-class/output/results/output_combineModel-ev2l-Keras-finetunning-ImgNetWeight-512')
    try:
        os.makedirs(output_test_dir, exist_ok=True)
        print("Directory created")
    except OSError as e:
        print("Directory already exists")
count = 0
countf = 0
countF = 0
countT = 0
count_loop = 0
count1 = 0
count2 = 0
count3 = 0
count4 = 0
count5 = 0
count6 = 0
count7 = 0
count8 = 0
count9 = 0
count10 = 0

root, class_n, files = next(os.walk(result_test_data_dir))
print(class_n)
for c in class_n:
    new_root = os.path.join(output_test_dir, c)
    try:
        os.mkdir(new_root)
    except OSError as e:
        print("Directory already exists")
    for dirs in os.walk(os.path.join(root, c)):
        for files in dirs:
            for filename in files:
                if(filename.endswith(".jpg")):
                    path = os.path.join(root, c, filename)
                    #img = tf.keras.utils.load_img(path, target_size=(img_height, img_width))
                    orig = cv2.imread(path)
                    img = cv2.cvtColor(orig, cv2.COLOR_BGR2RGB)
                    img = cv2.resize(img, (img_height, img_width))
                    img_array = tf.keras.utils.img_to_array(img)
                    img_array = tf.expand_dims(img_array, 0) # Create a batch
                    ##NO Need of normalization here
                    ##img_array = normalization_layer(img_array)
                    
                    with tf.device('device:GPU:0'):
                        predictions = reloaded.predict(img_array)
                        score = tf.nn.softmax(predictions[0])

                    if (int(class_names[np.argmax(score)])==int(c)):
                        count=count+1
                    elif(int(class_names[np.argmax(score)]) ==10):
                        countf=countf+1
                        count10=count10+1
                    elif(int(class_names[np.argmax(score)])==9):
                        countf=countf+1
                        count9=count9+1
                    elif(int(class_names[np.argmax(score)]) ==8):
                        countf=countf+1
                        count8=count8+1
                    elif(int(class_names[np.argmax(score)]) ==7):
                        countf=countf+1
                        count7=count7+1
                    elif(int(class_names[np.argmax(score)]) ==6):
                        countf=countf+1
                        count6=count6+1
                    elif(int(class_names[np.argmax(score)]) ==5):
                        countf=countf+1
                        count5=count5+1
                    elif(int(class_names[np.argmax(score)]) ==4):
                        countf=countf+1
                        count4=count4+1
                    elif(int(class_names[np.argmax(score)]) ==3):
                        countf=countf+1
                        count3=count3+1
                    elif(int(class_names[np.argmax(score)])==2):
                        countf=countf+1
                        count2=count2+1
                    elif(int(class_names[np.argmax(score)])==1):
                        countf=countf+1
                        count1=count1+1
                    else:
                        print('error')
                    new_filename = filename.split(".")[0]+ filename.split(".")[1] + "_" + "P_" + str(int(class_names[np.argmax(score)])) + ".jpg"
                    new_path = os.path.join(new_root, new_filename)
                    cv2.imwrite(new_path, orig)
                    #print(new_filename )
    countT = countT + count
    countF = countF + countf
    print('=========================================================')
    print('class: ',c, 'Images: ',len(files), 'TP: ', count/len(files), 'FP: ',countf/len(files))
    #print('one : ',count1/len(files),'two: ',count2/len(files),'three: ',count3/len(files),'four: ',count4/len(files),'five: ',count5/len(files),'six: ',count6/len(files),'seven: ',count7/len(files),'eight: ',count8/len(files),'nine: ',count9/len(files),'ten: ',count10/len(files))
    print('one: ',count1,'two: ',count2,'three: ',count3,'four: ',count4,'five: ',count5,'six: ',count6,'seven: ',count7,'eight: ',count8,'nine: ',count9,'ten: ',count10)
    test.write(int(c), 1, count1)
    test.write(int(c), 2, count2)
    test.write(int(c), 3, count3)
    test.write(int(c), 4, count4)
    test.write(int(c), 5, count5)
    test.write(int(c), 6, count6)
    test.write(int(c), 7, count7)
    test.write(int(c), 8, count8)
    test.write(int(c), 9, count9)
    test.write(int(c), 10, count10)
    test.write(int(c), 11, count)
    test.write(int(c), 12, countf)

    count = 0
    countf = 0
    count1 = 0
    count2 = 0
    count3 = 0
    count4 = 0
    count5 = 0
    count6 = 0
    count7 = 0
    count8 = 0
    count9 = 0
    count10 = 0
    count_loop = count_loop + 1
print('Total classes: ',count_loop,'Total TP: ', countT/(countT+countF), 'Total FP: ',countF)
test.write(11, 0, countT/(countT+countF))
test.write(11, 1, countF)

In [ ]:
wb.save('output_road__combined-road-Model-testData-ev2l-Keras-finetunning-ImgNetWeight-512.xls')

retrain the outer layers with more data from only road

In [ ]:
# Workbook is created
wb2 = Workbook()
# add_sheet is used to create sheet.
test2 = wb2.add_sheet('Test-Summary-road')
test2.write(0,0,'No')
test2.write(0,1,'filename')
test2.write(0,2,'true-class')
test2.write(0,3,'predicted-class')
iter = 1

with tf.device('device:GPU:0'):
    #result_test_data_dir = pathlib.Path('/home/pms/PMS-dataset/data-set/10-class/combined/testing')
    result_test_data_dir = pathlib.Path('/home/pms/PMS-dataset/data-set/10-class/output/testing')
    #output_test_dir = pathlib.Path('/home/pms/PMS-dataset/data-set/10-class/combined/results/test')
    output_test_dir = pathlib.Path('/home/pms/PMS-dataset/data-set/10-class/output/results/test')
    try:
        os.makedirs(output_test_dir, exist_ok=True)
        print("Directory created")
    except OSError as e:
        print("Directory already exists")
count = 0
countf = 0
countF = 0
countT = 0
count_loop = 0
count1 = 0
count2 = 0
count3 = 0
count4 = 0
count5 = 0

root, class_n, files = next(os.walk(result_test_data_dir))
print(class_n)
for c in class_n:
    new_root = os.path.join(output_test_dir, c)
    try:
        os.mkdir(new_root)
    except OSError as e:
        print("Directory already exists")
    for dirs in os.walk(os.path.join(root, c)):
        for files in dirs:
            for filename in files:
                if(filename.endswith(".jpg")):
                    path = os.path.join(root, c, filename)
                    #img = tf.keras.utils.load_img(path, target_size=(img_height, img_width))
                    orig = cv2.imread(path)
                    img = cv2.cvtColor(orig, cv2.COLOR_BGR2RGB)
                    img = cv2.resize(img, (img_height, img_width))
                    img_array = tf.keras.utils.img_to_array(img)
                    img_array = tf.expand_dims(img_array, 0) # Create a batch
                    #-------------------------------
                    ##NO NORMALIZATION REQUIRED
                    ##img_array = normalization_layer(img_array)
                    #-------------------------------
                    predictions = reloaded.predict(img_array)
                    score = tf.nn.softmax(predictions[0])
                    ## write to excel------------------------------------
                    test2.write(iter, 0, iter)
                    test2.write(iter, 1, filename)
                    test2.write(iter, 2, int(c))
                    test2.write(iter, 3, int(class_names[np.argmax(score)]))
                    iter = iter + 1 # this iterator is for serial number in excel file to count number of jpg files
                        #-----------------------------------------------------
                    if (int(class_names[np.argmax(score)])==int(c)):
                        count=count+1
                    elif(int(class_names[np.argmax(score)]) ==5):
                        countf=countf+1
                        count5=count5+1
                    elif(int(class_names[np.argmax(score)]) ==4):
                        countf=countf+1
                        count4=count4+1
                    elif(int(class_names[np.argmax(score)]) ==3):
                        countf=countf+1
                        count3=count3+1
                    elif(int(class_names[np.argmax(score)])==2):
                        countf=countf+1
                        count2=count2+1
                    elif(int(class_names[np.argmax(score)])==1):
                        countf=countf+1
                        count1=count1+1
                    elif(int(class_names[np.argmax(score)])==6):
                        countf=countf+1
                        count6=count6+1
                    elif(int(class_names[np.argmax(score)])==7):
                        countf=countf+1
                        count7=count7+1
                    elif(int(class_names[np.argmax(score)])==8):
                        countf=countf+1
                        count8=count8+1
                    elif(int(class_names[np.argmax(score)])==9):
                        countf=countf+1
                        count9=count9+1
                    elif(int(class_names[np.argmax(score)])==10):
                        countf=countf+1
                        count10=count10+1
                    else:
                        print('error')
                    new_filename = filename.split(".")[0]+ filename.split(".")[1] + "_" + "P_" + str(int(class_names[np.argmax(score)])) + ".jpg"
                    new_path = os.path.join(new_root, new_filename)
                    cv2.imwrite(new_path, orig)
                    #print(new_filename )

    countT = countT + count
    countF = countF + countf
    print('=========================================================')
    print('class: ',c, 'Images: ',len(files), 'TP: ', count/len(files), 'FP: ',countf/len(files))
    print('one: ',count1,'two: ',count2,'three: ',count3,'four: ',count4,'five: ',count5)

    count = 0
    countf = 0
    count1 = 0
    count2 = 0
    count3 = 0
    count4 = 0
    count5 = 0
    count6 = 0
    count7 = 0
    count8 = 0
    count9 = 0
    count10 = 0
    count_loop = count_loop + 1
print('Total classes: ',count_loop,'Total TP: ', countT/(countT+countF), 'Total FP: ',countF)

wb2.save('road-10-classes-finetune.xls')